In [ ]:
import numpy as np
import sys

In [ ]:
from misc import load, load_class_labels

dataset_root = "../dataset/"

train_dir = dataset_root + "train_set/"
val_dir = dataset_root + "val_set/"
val_degraded_dir = dataset_root + "val_set_degraded/"

sample_rate = 1
train_info = load(dataset_root + 'labels/train.csv', 1, sample_rate)
val_info = load(dataset_root + 'labels/validation.csv')
test_info = load(dataset_root + 'labels/test.csv')

train_image_names = train_info.iloc[:, 0].values
val_image_names = val_info.iloc[:, 0].values

train_labels = train_info.iloc[:, 1].values
val_labels = val_info.iloc[:, 1].values

class_labels = load_class_labels(dataset_root + 'classes.txt')

In [ ]:
from nn_filter import NN_filter

filter = NN_filter(train_info, train_dir, 'efficient_net', batch_size=25)

In [ ]:
features, labels, image_names = filter.fit_train()

## Extract validation from trainig set

In [ ]:
import pandas as pd
train_info = load(dataset_root + 'train_info_dirty.csv', 1, 0.8)
val_info = load(dataset_root + 'train_info_dirty.csv', 1, 1)


same = val_info['filename'].isin(train_info['filename'])
val_info.drop(val_info[same].index, inplace=True)

#train_info.to_csv(dataset_root + '/labels/train.csv', header=False, index=False);
#val_info.to_csv(dataset_root + '/labels/validation.csv', header=False, index=False);

## Outlier detection with NN features

In [ ]:
from image_discard import discard_preprocessing
names, indices = discard_preprocessing(train_dir, train_image_names, train_labels)

In [ ]:
count = 0
for c in names:
    count += len(c)
count

In [ ]:
kept_indices = ~np.isin([i for i in range(len(train_image_names))], indices)

preprocessed_features = features[kept_indices]
preprocessed_train_names = train_image_names[kept_indices]
preprocessed_train_labels = train_labels[kept_indices]
preprocessed_train_info = pd.DataFrame({
    'filename': preprocessed_train_names,
    'label': preprocessed_train_labels
})

In [ ]:
from image_discard import write_discarded_images
write_discarded_images(names, class_labels, dataset_root + "discarded_preprocess/", train_dir)

In [ ]:
from image_discard import find_outliers_iter_median
names, distances = find_outliers_iter_median(preprocessed_features, preprocessed_train_labels, preprocessed_train_names, threshold=0.95)

In [ ]:
from image_discard import write_discarded_images, find_outliers_iter, write_cleaned_csv
write_discarded_images(names, class_labels, dataset_root + "discarded_scaling_features_efficientnetv2_median/", train_dir)

In [ ]:
clean_train_info = write_cleaned_csv(preprocessed_train_info, names, dataset_root + "clean_efficientnet_v2/median/")

In [ ]:
clean_names = clean_train_info.iloc[:, 0].values
clean_labels = clean_train_info.iloc[:, 1].values

kept_names = [[] for _ in range(251)]
for name, label in zip(clean_names, clean_labels):
    kept_names[label].append(name)

write_discarded_images(kept_names, class_labels, dataset_root + "kept_efficientnet_v2_median/", train_dir)